# 📊 Agent Telemetry & Tracing for Wealth Management Advisory

This notebook demonstrates how to implement **telemetry and tracing** for AI agents using Azure Monitor and OpenTelemetry. We'll build a **Wealth Management Advisory Agent** that provides investment guidance while capturing detailed traces for monitoring, debugging, and compliance.

## 🎯 Learning Objectives

1. **Configure Azure Monitor** for agent telemetry
2. **Enable content recording** to capture prompts and responses
3. **Create custom trace spans** for business-specific context
4. **View traces** in Microsoft Foundry and Application Insights
5. **Understand trace data** for compliance and debugging

## 💼 Industry Use Case: Wealth Management Advisory

In wealth management, telemetry is critical for:
- **Regulatory Compliance**: Track all advisor-client interactions
- **Audit Trail**: Maintain records of investment recommendations
- **Performance Monitoring**: Measure response times and quality
- **Risk Management**: Detect anomalies in agent behavior

### ⚠️ Financial Disclaimer
> **The financial information provided is for educational purposes only and is not investment advice.** Always consult with qualified financial advisors before making investment decisions.

<img src="./seq-diagrams/1-observability.png" width="75%"/>

## 🔐 Authentication Setup

Before running this notebook, authenticate with Azure CLI:

```bash
az login --use-device-code
```

This provides browser-based authentication useful for remote environments and corporate security policies.

## 1. Environment Setup & Configuration

We'll load environment variables and set up the telemetry configuration **before** creating any clients.

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file
notebook_path = Path().absolute()
env_path = notebook_path.parent / '.env'
load_dotenv(env_path)

# Verify required environment variables
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
tenant_id = os.environ.get("TENANT_ID")
model_deployment = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

if not project_endpoint:
    raise ValueError("🚨 AI_FOUNDRY_PROJECT_ENDPOINT not set in .env")

print(f"🔑 Tenant ID: {tenant_id}")
print(f"📍 Project Endpoint: {project_endpoint[:50]}...")
print(f"🤖 Model Deployment: {model_deployment}")

## 2. Enable Content Recording for Telemetry

**Important**: Set the environment variable to capture message content (prompts and responses) **before** any instrumentation. This is essential for compliance auditing in FSI.

⚠️ **Privacy Note**: Content recording may capture personal data. Use with caution in production.

In [ ]:
# Enable content recording BEFORE any instrumentation
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

print("✅ Content recording enabled for telemetry")
print("📝 Prompts and responses will be captured in traces")

## 3. Configure Azure Monitor Tracing

Azure Monitor with OpenTelemetry provides:
- Distributed tracing across agent calls
- Performance metrics and latency tracking
- Integration with Application Insights
- Visualization in Microsoft Foundry portal

In [ ]:
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient
from opentelemetry import trace

# First, create a temporary client to get the Application Insights connection string
temp_credential = AzureCliCredential(tenant_id=tenant_id)
temp_client = AIProjectClient(endpoint=project_endpoint, credential=temp_credential)

# Get Application Insights connection string from the AI Project
# Using the correct method from Azure SDK samples
app_insights_connection = os.environ.get("APPLICATIONINSIGHTS_CONNECTION_STRING")

if not app_insights_connection:
    # Get connection string from AI Project telemetry (correct method name)
    try:
        app_insights_connection = temp_client.telemetry.get_application_insights_connection_string()
        print(f"✅ Retrieved Application Insights connection from AI Project")
    except Exception as e:
        print(f"⚠️ Could not get connection string from project: {e}")

# Configure Azure Monitor for tracing
if app_insights_connection:
    configure_azure_monitor(connection_string=app_insights_connection)
    print(f"✅ Azure Monitor configured with Application Insights")
else:
    print("⚠️ No Application Insights connection - traces will not be exported")
    print("   Add APPLICATIONINSIGHTS_CONNECTION_STRING to .env or configure in AI Foundry")

# Get a tracer for creating custom spans
tracer = trace.get_tracer("wealth-management-advisor")
print(f"✅ Tracer initialized: wealth-management-advisor")

## 4. Initialize AI Project Client with Tracing

Now we create the AIProjectClient. With Azure Monitor configured, all agent calls will be automatically traced.

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition

# Reuse the client created during telemetry setup (with tracing now configured)
project_client = temp_client
credential = temp_credential

print("✅ AIProjectClient ready with tracing enabled")

## 5. Create Wealth Management Advisory Agent

We'll create an agent specialized in wealth management that provides investment guidance with appropriate disclaimers. The agent creation and all interactions will be traced.

In [ ]:
def create_wealth_advisor_agent():
    """Create a wealth management advisory agent with compliance-focused instructions."""
    
    # Create a custom span for agent creation
    with tracer.start_as_current_span("create_wealth_advisor") as span:
        span.set_attribute("agent.type", "wealth_management")
        span.set_attribute("agent.model", model_deployment)
        
        try:
            agent = project_client.agents.create_version(
                agent_name="wealth-management-advisor",
                definition=PromptAgentDefinition(
                    model=model_deployment,
                    instructions="""
                    You are a Wealth Management Advisory Assistant for high-net-worth clients.
                    
                    Your responsibilities:
                    1. Provide educational information about investment strategies, asset allocation, and portfolio diversification.
                    2. Explain financial concepts like risk tolerance, market volatility, tax-efficient investing, and estate planning.
                    3. Discuss different asset classes: equities, fixed income, alternatives, and real estate.
                    4. Always include appropriate disclaimers about investment risks.
                    5. Recommend consulting with a licensed financial advisor for personalized advice.
                    6. Never provide specific stock picks or guarantee investment returns.
                    7. Consider client risk profiles when discussing investment options.
                    
                    IMPORTANT DISCLAIMERS:
                    - Past performance does not guarantee future results
                    - All investments carry risk including potential loss of principal
                    - This is educational information, not personalized investment advice
                    """
                )
            )
            
            # Add trace attributes for successful creation
            span.set_attribute("agent.name", agent.name)
            span.set_attribute("agent.version", agent.version)
            span.set_attribute("agent.status", "created")
            
            print(f"🎉 Created Wealth Management Advisor (name: {agent.name}, version: {agent.version})")
            return agent
            
        except Exception as e:
            span.set_attribute("agent.status", "failed")
            span.set_attribute("error.message", str(e))
            print(f"❌ Error creating agent: {e}")
            return None

# Create the wealth advisor agent
wealth_advisor = create_wealth_advisor_agent()

## 6. Chat Function with Custom Tracing

We'll create a chat function that adds business-specific trace attributes for compliance and monitoring. Each client interaction will include:
- Client profile information
- Query category (portfolio, retirement, tax, etc.)
- Response metrics

In [ ]:
import time

def chat_with_wealth_advisor(user_question: str, client_id: str = "CLIENT-001", query_category: str = "general"):
    """
    Chat with the wealth advisor agent with full tracing.
    
    Args:
        user_question: The client's question
        client_id: Client identifier for compliance tracking
        query_category: Category of query (portfolio, retirement, tax, risk, general)
    """
    
    if not wealth_advisor:
        print("❌ No agent available. Please create an agent first.")
        return None
    
    # Create a custom span for the advisory interaction
    with tracer.start_as_current_span("wealth_advisory_interaction") as span:
        # Add business-specific attributes for compliance
        span.set_attribute("client.id", client_id)
        span.set_attribute("query.category", query_category)
        span.set_attribute("query.length", len(user_question))
        span.set_attribute("agent.name", wealth_advisor.name)
        
        start_time = time.time()
        
        try:
            print(f"👤 Client [{client_id}]: {user_question}")
            print(f"📂 Category: {query_category}")
            print("🔄 Processing...")
            
            # Get the OpenAI client from project client
            openai_client = project_client.get_openai_client()
            
            # Call the agent - this is automatically traced
            response = openai_client.responses.create(
                extra_body={
                    "agent": {
                        "type": "agent_reference",
                        "name": wealth_advisor.name,
                        "version": wealth_advisor.version
                    }
                },
                input=user_question
            )
            
            # Calculate response time
            response_time = time.time() - start_time
            
            # Add response metrics to trace
            span.set_attribute("response.time_ms", int(response_time * 1000))
            span.set_attribute("response.status", "success")
            
            if response.output_text:
                span.set_attribute("response.length", len(response.output_text))
                print(f"\n🤖 Advisor: {response.output_text}")
                print(f"\n⏱️ Response time: {response_time:.2f}s")
                return response.output_text
            else:
                span.set_attribute("response.status", "empty")
                print("❌ No response from advisor")
                return None
                
        except Exception as e:
            response_time = time.time() - start_time
            span.set_attribute("response.time_ms", int(response_time * 1000))
            span.set_attribute("response.status", "error")
            span.set_attribute("error.message", str(e))
            print(f"❌ Error: {e}")
            return None

print("✅ Chat function with tracing defined")

## 7. Test Advisory Interactions with Tracing

Let's run several advisory interactions. Each will generate traces that you can view in Microsoft Foundry or Application Insights.

In [ ]:
print("🧪 Testing Wealth Management Advisory with Telemetry")
print("=" * 60)

# Test 1: Portfolio Allocation Question
print("\n📊 Test 1: Portfolio Allocation")
print("-" * 40)
response1 = chat_with_wealth_advisor(
    user_question="I'm 45 years old with a moderate risk tolerance. How should I think about allocating my portfolio between stocks and bonds?",
    client_id="HNW-2024-001",
    query_category="portfolio"
)

In [ ]:
# Test 2: Retirement Planning Question
print("\n🏖️ Test 2: Retirement Planning")
print("-" * 40)
response2 = chat_with_wealth_advisor(
    user_question="What's the difference between a Traditional IRA and a Roth IRA? Which might be better for someone in a high tax bracket?",
    client_id="HNW-2024-002",
    query_category="retirement"
)

In [ ]:
# Test 3: Tax-Efficient Investing
print("\n💰 Test 3: Tax-Efficient Investing")
print("-" * 40)
response3 = chat_with_wealth_advisor(
    user_question="What strategies can help minimize taxes on investment gains? I have both taxable and tax-advantaged accounts.",
    client_id="HNW-2024-003",
    query_category="tax"
)

In [ ]:
# Test 4: Risk Assessment
print("\n⚠️ Test 4: Risk Assessment")
print("-" * 40)
response4 = chat_with_wealth_advisor(
    user_question="With current market volatility, should I be concerned about my equity exposure? How do I evaluate if my portfolio risk is appropriate?",
    client_id="HNW-2024-001",  # Same client, different query
    query_category="risk"
)

## 8. Retrieve and Display Trace Information

You can view traces in multiple places:

1. **Microsoft Foundry Portal**: Navigate to your project > Tracing
2. **Application Insights**: Transaction Search > Filter by operation name
3. **Programmatically**: Using the OpenTelemetry SDK

Let's get the current trace context:

In [ ]:
from opentelemetry import trace
from opentelemetry.trace import format_trace_id, format_span_id

# Create a sample span to demonstrate trace ID retrieval
with tracer.start_as_current_span("trace_info_demo") as span:
    context = span.get_span_context()
    trace_id = format_trace_id(context.trace_id)
    span_id = format_span_id(context.span_id)
    
    print("📊 Current Trace Information")
    print("=" * 50)
    print(f"Trace ID: {trace_id}")
    print(f"Span ID: {span_id}")
    print(f"\n🔍 View traces in Microsoft Foundry:")
    print(f"   1. Open your AI Foundry project")
    print(f"   2. Navigate to 'Tracing' section")
    print(f"   3. Search for trace ID: {trace_id}")
    print(f"\n📈 View in Application Insights:")
    print(f"   1. Open Application Insights in Azure Portal")
    print(f"   2. Go to 'Transaction search'")
    print(f"   3. Filter by 'wealth-management-advisor'")

## 9. Test Summary

Let's summarize the tests and telemetry captured:

In [ ]:
print("\n" + "=" * 60)
print("📊 TELEMETRY TEST SUMMARY")
print("=" * 60)

tests = [
    ("Portfolio Allocation", response1),
    ("Retirement Planning", response2),
    ("Tax-Efficient Investing", response3),
    ("Risk Assessment", response4)
]

successful = sum(1 for _, r in tests if r is not None)

print(f"\n✅ Successful interactions: {successful}/4")
print(f"\n📝 Telemetry captured for each interaction:")
print("   • Client ID (for compliance tracking)")
print("   • Query category (portfolio, retirement, tax, risk)")
print("   • Response time (performance monitoring)")
print("   • Full prompt and response content (audit trail)")
print("   • Error details (if any occurred)")

print(f"\n🎯 FSI Compliance Benefits:")
print("   • Complete audit trail of client interactions")
print("   • Regulatory reporting capabilities")
print("   • Performance SLA monitoring")
print("   • Anomaly detection for risk management")

## 10. Cleanup

Optionally delete the agent when finished. In production, you'd typically keep agents running.

In [ ]:
# Uncomment to delete the agent
# if wealth_advisor:
#     project_client.agents.delete_version(
#         agent_name=wealth_advisor.name, 
#         agent_version=wealth_advisor.version
#     )
#     print("🗑️ Deleted Wealth Management Advisor agent")

print("✅ Notebook completed - Agent retained for further use")

## 🎯 Summary

In this notebook, you learned how to:

✅ **Configure Azure Monitor** for agent telemetry with OpenTelemetry  
✅ **Enable content recording** to capture prompts and responses for compliance  
✅ **Create custom trace spans** with business-specific attributes  
✅ **Build a Wealth Management Advisor** with full observability  
✅ **Track client interactions** with compliance-ready telemetry  

### 🔧 Key APIs Used

| API | Purpose |
|-----|--------|
| `configure_azure_monitor()` | Set up Azure Monitor exporter |
| `trace.get_tracer()` | Get a tracer for custom spans |
| `tracer.start_as_current_span()` | Create custom trace spans |
| `span.set_attribute()` | Add business context to traces |
| `format_trace_id()` | Get readable trace ID for debugging |

### 📚 Next Steps

1. **View traces** in Microsoft Foundry portal or Application Insights
2. **Set up alerts** for anomalous response times or error rates
3. **Create dashboards** for compliance reporting
4. **Explore evaluation** in `2-evaluation.ipynb` to assess agent quality

### 📖 Related Resources

- [Microsoft Foundry Observability](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability)
- [Azure Monitor OpenTelemetry](https://learn.microsoft.com/en-us/azure/azure-monitor/app/opentelemetry-overview)
- [OpenTelemetry Python](https://opentelemetry.io/docs/instrumentation/python/)